In [2]:
pip install kiwipiepy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# 1. 필요한 라이브러리 임포트
import json
import re
from kiwipiepy import Kiwi
from tqdm import tqdm
from pprint import pprint as pp
import pandas as pd

# JSON 파일 로딩
with open("C:/Users/gram/KORE208_team5/이준석.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# data는 리스트 형태
contents = [article.get("content", "") for article in data]

In [11]:
pp(type(data))

<class 'list'>


In [16]:
from pprint import pprint as pp

pp(data[0].keys())   # 리스트 안 첫 번째 요소의 키 확인


dict_keys(['id', 'order', 'title', 'press', 'date', 'content', 'source'])


In [17]:
for i, item in enumerate(data):
    pp(f"index : {i}; type: {type(item)}; length: {len(item)}")

"index : 0; type: <class 'dict'>; length: 7"
"index : 1; type: <class 'dict'>; length: 7"
"index : 2; type: <class 'dict'>; length: 7"
"index : 3; type: <class 'dict'>; length: 7"
"index : 4; type: <class 'dict'>; length: 7"
"index : 5; type: <class 'dict'>; length: 7"
"index : 6; type: <class 'dict'>; length: 7"
"index : 7; type: <class 'dict'>; length: 7"
"index : 8; type: <class 'dict'>; length: 7"
"index : 9; type: <class 'dict'>; length: 7"
"index : 10; type: <class 'dict'>; length: 7"
"index : 11; type: <class 'dict'>; length: 7"
"index : 12; type: <class 'dict'>; length: 7"
"index : 13; type: <class 'dict'>; length: 7"
"index : 14; type: <class 'dict'>; length: 7"
"index : 15; type: <class 'dict'>; length: 7"
"index : 16; type: <class 'dict'>; length: 7"
"index : 17; type: <class 'dict'>; length: 7"
"index : 18; type: <class 'dict'>; length: 7"
"index : 19; type: <class 'dict'>; length: 7"
"index : 20; type: <class 'dict'>; length: 7"
"index : 21; type: <class 'dict'>; length: 7

In [18]:
# 3. 텍스트 전처리 함수 정의
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)                 # HTML 태그 제거
    text = re.sub(r'\n+', ' ', text)                  # 줄바꿈 -> 공백
    text = re.sub(r'["“”‘’\'!?…·,;:\(\)\[\]\{\}]', '', text)  # 주요 문장부호 제거
    text = re.sub(r'[^\w\s가-힣]', '', text)           # 그 외 특수문자 제거 (이중 필터링)
    text = re.sub(r'\s+', ' ', text).strip()          # 다중 공백 제거
    return text

cleaned_contents = [clean_text(text) for text in contents]



In [31]:
# 4. 문장 단위로 품사 태깅 결과를 병합

# 형태소 분석기 초기화
kiwi = Kiwi()

# 사용자 정의 사전 추가
user_words = ['개혁신당', '민주당', '국민의힘', '아류', '남소연']
for word in user_words:
    kiwi.add_user_word(word, 'NNP', 0)

rows = []
for doc_id, text in enumerate(cleaned_contents):
    sentences = kiwi.split_into_sents(text)
    for sent in sentences:
        analyzed = kiwi.analyze(sent.text)[0][0]
        tagged_str = ' '.join([f"{token.form}({token.tag})" for token in analyzed])
        rows.append({
            "doc_id": doc_id,
            "original_sentence": sent.text,
            "tagged_sentence": tagged_str
        })

# 5. DataFrame 생성
df_tagged = pd.DataFrame(rows)


In [32]:
# 4. 분석에 의미 없는 단어 기반 불용어 리스트
stopwords = set([
    "것", "수", "등", "더", "때", "자신", "이번", "이후", "위해", "관련",
    "의견", "최근", "대해", "내용", "경우", "부분", "정도",
    "현재", "상황", "사실", "또한", "통해", "가장", "대한", "중요", "전혀",
    "하나", "바로", "계속", "여러", "많은", "이미"
])

# 5. 제거할 불필요한 품사 리스트
stop_tags = {
    'JKS', 'JKC', 'JKG', 'JKB', 'JKO', 'JKV', 'JKQ',   # 조사
    'JX', 'JC',                                # 보조사, 접속조사
    'NNB',                                     # 의존명사
    'NR', 'SN',                                # 수사, 숫자
    'SF', 'SP', 'SS', 'SE', 'SO',              # 구두점
    'EP', 'EF', 'EC', 'ETN', 'ETM',            # 어미
    'XPN', 'XSN', 'XSV', 'XSA',                # 접사
    'IC',                                      # 감탄사
    'SF', 'SE', 'SS', 'SP', 'SO',              # 기호
    'VSV', 'NR'                                # 기타 불분명 품사
}

In [33]:
# 7. 형태소 분석 + 불용품사 제거 + 불용어 제거
tagged_sentences = []

for text in tqdm(cleaned_contents):
    analyzed = kiwi.analyze(text)[0][0]
    filtered = [
        f"{word}({tag})" for word, tag, _, _ in analyzed
        if tag not in stop_tags and word not in stopwords
    ]
    tagged_sentences.append(" ".join(filtered))

# 8. 결과 저장

articles = data

df = pd.DataFrame({
    "title": [article["title"] for article in articles],
    "date": [article["date"] for article in articles],
    "original": contents,
    "cleaned": cleaned_contents,
    "tagged": tagged_sentences
})



100%|██████████| 1000/1000 [00:37<00:00, 26.32it/s]


In [35]:
df.to_csv("lee_junseok_tagged_filtered_ver2.csv", index=False, encoding="utf-8-sig")